In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers

In [20]:
IMAGE_SIZE = (128, 128)
NUM_CLASSES = 2 

DATA_DIR = r'C:\Users\geeth\Downloads\test_data\YOLO-Object-Detection\DL project\Data'
DEFECTIVE_DIR = os.path.join(DATA_DIR, 'Defective_cubes')
QUALITY_DIR = os.path.join(DATA_DIR, 'Quality_cubes')

In [21]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [4]:
def load_data(directory):
    images = []
    labels = []
    label = 0 if directory == QUALITY_DIR else 1  # Assign label based on directory
    for filename in os.listdir(directory):
        if filename.lower().endswith('.jpg'):
            img_path = os.path.join(directory, filename)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMAGE_SIZE)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)


In [5]:
defective_images, defective_labels = load_data(DEFECTIVE_DIR)
quality_images, quality_labels = load_data(QUALITY_DIR)

In [6]:
cube_images = np.concatenate([defective_images, quality_images], axis=0)
cube_labels = np.concatenate([defective_labels, quality_labels], axis=0)

train_images, test_images, train_labels, test_labels = train_test_split(cube_images, cube_labels, test_size=0.2, random_state=42)


In [7]:
train_images_normalized = train_images / 255.0
test_images_normalized = test_images / 255.0

In [8]:
def precision(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f1_score = 2 * (p * r) / (p + r + tf.keras.backend.epsilon())
    return f1_score

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='selu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='selu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='selu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(256, activation='selu', kernel_regularizer=regularizers.l2(0.1)),  # Apply L2 regularization
    tf.keras.layers.BatchNormalization(),  # Add BatchNormalization
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')  # Binary classification
])

In [23]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [24]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)


In [25]:
model.fit(datagen.flow(train_images_normalized, train_labels, batch_size=32),
          epochs=10, 
          validation_data=(test_images_normalized, test_labels),
          callbacks=[early_stopping, reduce_lr])

Epoch 1/10
112/112 [==============================] - 64s 541ms/step - loss: 3.3256 - accuracy: 0.6507 - val_loss: 2.6762 - val_accuracy: 0.5709 - lr: 0.0010
Epoch 2/10
112/112 [==============================] - 58s 512ms/step - loss: 1.1267 - accuracy: 0.6695 - val_loss: 9.8703 - val_accuracy: 0.4950 - lr: 0.0010
Epoch 3/10
112/112 [==============================] - 61s 538ms/step - loss: 1.0674 - accuracy: 0.6706 - val_loss: 5.5274 - val_accuracy: 0.5274 - lr: 0.0010
Epoch 4/10
112/112 [==============================] - 59s 522ms/step - loss: 0.7409 - accuracy: 0.6812 - val_loss: 24.3153 - val_accuracy: 0.4894 - lr: 2.0000e-04


In [26]:
loss, accuracy = model.evaluate(test_images_normalized, test_labels)

28/28 [==============================] - 2s 79ms/step - loss: 2.6762 - accuracy: 0.5709
